In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization, Reshape, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

# Rastgele gürültü (noise) üretmek için yardımcı fonksiyon
def generate_noise(batch_size, noise_dim):
    return np.random.normal(0, 1, (batch_size, noise_dim))

# Üretici (generator) ağının tanımlanması
def build_generator(noise_dim):
    model = Sequential()
    model.add(Dense(256, input_dim=noise_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(28*28*1, activation='tanh'))
    model.add(Reshape((28, 28, 1)))
    return model

# Ayrımcı (discriminator) ağının tanımlanması
def build_discriminator(img_shape):
    model = Sequential()
    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    return model

# GAN modelinin tanımlanması
def build_gan(generator, discriminator):
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

# Hiperparametreler
img_shape = (28, 28, 1)
noise_dim = 100
batch_size = 64
epochs = 10000

# Ağların inşa edilmesi
generator = build_generator(noise_dim)
discriminator = build_discriminator(img_shape)

# Ayrımcının derlenmesi
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])

# Ayrımcının eğitilmesini dondur
discriminator.trainable = False

# GAN modelinin derlenmesi
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))

# Eğitim fonksiyonu
def train(epochs, batch_size):
    (X_train, _), (_, _) = tf.keras.datasets.mnist.load_data()
    X_train = X_train / 127.5 - 1.0
    X_train = np.expand_dims(X_train, axis=3)
    real = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]
        noise = generate_noise(batch_size, noise_dim)
        gen_imgs = generator.predict(noise)

        d_loss_real = discriminator.train_on_batch(imgs, real)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        noise = generate_noise(batch_size, noise_dim)
        g_loss = gan.train_on_batch(noise, real)

        if epoch % 1000 == 0:
            print(f"{epoch} [D loss: {d_loss[0]} | D acc.: {100 * d_loss[1]}] [G loss: {g_loss}]")
            save_imgs(epoch)

# Üretilen görsellerin kaydedilmesi
def save_imgs(epoch):
    noise = generate_noise(25, noise_dim)
    gen_imgs = generator.predict(noise)
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(5, 5)
    cnt = 0
    for i in range(5):
        for j in range(5):
            axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
            axs[i, j].axis('off')
            cnt += 1
    fig.savefig(f"mnist_{epoch}.png")
    plt.close()

# Modelin eğitilmesi
train(epochs, batch_size)


Streaming output truncated to the last 5000 lines.
2/2 [==============================] - 0s 12ms/step
6000 [D loss: 0.6540183126926422 | D acc.: 65.625] [G loss: 0.9005138278007507]
2/2 [==============================] - 0s 13ms/step
7000 [D loss: 0.6419790685176849 | D acc.: 59.375] [G loss: 0.8760648965835571]
2/2 [==============================] - 0s 13ms/step
8000 [D loss: 0.6445170938968658 | D acc.: 61.71875] [G loss: 0.9580690860748291]
2/2 [==============================] - 0s 12ms/step
9000 [D loss: 0.6296452283859253 | D acc.: 65.625] [G loss: 0.9079599976539612]
2/2 [==============================] - 0s 12ms/step
